In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
import re
import os

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer

In [5]:
os.listdir("./data")

['used-cars.csv', '.gitignore']

In [6]:
raw = pd.read_csv("./data/used-cars.csv")

In [7]:
# initial cleanup
cars = raw.copy()
cars.drop("Unnamed: 0", axis=1, inplace=True)
cars.columns = [x.lower() for x in cars.columns]

In [8]:
cars.head()

,name,location,year,kilometers_driven,fuel_type,transmission,owner_type,mileage,engine,power,seats,new_price,price
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


In [56]:
# will be passed a dataframe and should return a dataframe

def car_make(df):
    return df.apply(lambda x: x.str.split(" ").str[0])

def remove_units(df):
    return df.apply(lambda x: x.str.split(" ").str[0].astype("float"))

def convert_years(df):
    return df.apply(lambda x: 2019 - x)

def bin_owner(df):
    return df.map(lambda x: "Third" if x not in ["First", "Second"] else x)

def ordinal_owner(df):
    owner_map = {
        "First": 1,
        "Second": 2,
        "Third": 3
    }
    
    return df.map(lambda x: owner_map[x])

In [76]:
owner_pipeline = Pipeline(
    [
        ('high_val_bin', FunctionTransformer(bin_owner)),
        ('ordinal_encode', FunctionTransformer(ordinal_owner))
    ]
)

car_make_pipeline = Pipeline(
    [
        ('car_make', FunctionTransformer(car_make)),
        ('one_hot', OneHotEncoder())
    ]
)

cols_with_units_pipeline = Pipeline(
    [
        ("remove_units", FunctionTransformer(remove_units)),
        ("standard_scaler", StandardScaler())
    ]
)

ct = ColumnTransformer(
    [
         # seems to only work when passed a list of column names and expects function to take and return a dataframe
        ("owner_pipeline", owner_pipeline, ["owner_type"]),
        ("one_hot", OneHotEncoder(handle_unknown="ignore"), ["location", "fuel_type"]),
        ("cols_with_units", cols_with_units_pipeline, ["mileage", "engine", "power"]),
        ("ordinal", OrdinalEncoder(), ["transmission"]),
        ("convert_years", FunctionTransformer(convert_years), ["year"]),
        ("car_make_pipeline", car_make_pipeline, ["name"]),
    ],
    # remainder='passthrough'
)

pipe = Pipeline(
        [
            ("column transformers", ct),
            ("regression", LinearRegression()),
        ]
)

In [77]:
X_train = cars.drop(["new_price", "price"], axis = 1)
y_train = cars["price"]

In [79]:
pipe.fit_transform(X_train)

<5847x50 sparse matrix of type '<class 'numpy.float64'>'
	with 50810 stored elements in Compressed Sparse Row format>

In [51]:
scores = cross_val_score(pipe, X_train, y_train, cv=5)

/var/folders/wy/6r_pbb_j6_ncdv3syrg7ppwc0000gn/T/ipykernel_72796/4050945880.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: "Third" if x not in ["First", "Second"] else x)
/var/folders/wy/6r_pbb_j6_ncdv3syrg7ppwc0000gn/T/ipykernel_72796/4050945880.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: owner_map[x])
/var/folders/wy/6r_pbb_j6_ncdv3syrg7ppwc0000gn/T/ipykernel_72796/4050945880.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: "Third" if x not in ["First", "Second"] else x)
/var/folders/wy/6r_pbb_j6_ncdv3syrg7ppwc0000gn/T/ipykernel_72796/4050945880.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: owner_map[x])
/var/folders/wy/6r_pbb_j6_ncdv3syrg7ppwc0000gn/T/ipykernel_72796/4050945880.py:13: FutureWar

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/calebcrouse/venvs/basic/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py", line 681, in _fit_transform
    return Parallel(n_jobs=self.n_jobs)(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/calebcrouse/venvs/basic/lib/python3.12/site-packages/sklearn/utils/parallel.py", line 65, in __call__
    return super().__call__(iterable_with_config)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/calebcrouse/venvs/basic/lib/python3.12/site-packages/joblib/parallel.py", line 1863, in __call__
    return output if self.return_generator else list(output)
                                        

In [98]:
# one hot encode catagorical variables